In [ ]:
import segno
from PIL import Image, ImageChops, ImageDraw, ImageFont
import json
import io


ppi = 200
inch_per_cm = 0.393701

cm_to_px = lambda x: int(x*ppi*inch_per_cm)


In [ ]:

def create_qr(data, height=1):
    qrcode = segno.make(data, boost_error=False, micro=False)
    out = io.BytesIO()
    qrcode.save(out, scale=height, border=0, kind='png')
    out.seek(0)
    local_img = Image.open(out)
    print(local_img)
    return local_img
    

def save_img(img, path, printer:str="joost_thuis"):
    with open(f'./printer_data/{printer}.json', 'r') as f:
        printer_data = json.load(f)

    offset_img = ImageChops.y1(
        img, 
        cm_to_px(-printer_data["MARGIN_LEFT"]), 
        cm_to_px(-printer_data["MARGIN_TOP"])
    )

    offset_img.save(path)
    


# student: id, test: exam id, question id


In [22]:

def create_answer_box(img, y1, height, qr_code_text='q123456'):
    offset_x = cm_to_px(1.5)
    
    qr_code = create_qr(qr_code_text, 1)
    qr_size = cm_to_px(1.5)

    
    p1 = (offset_x)-4, y1
    p1_rect = (offset_x)+qr_size, y1
    p2 = img.size[0] - offset_x+1 -5, y1+height
    p2_rect = img.size[0] - offset_x  - 10, y1+height + qr_size

    qr_code = qr_code.resize((qr_size, qr_size))

    img.paste(qr_code, p1)
    img.paste(qr_code, p2)
    
    draw = ImageDraw.Draw(img)
    draw.rectangle([p1_rect, p2_rect], width=5, outline='black')
    
    
    
    return img

img = Image.new('RGB', (int(8.25*ppi), int(11.75*ppi)), color='white')

for i in range(3):
    create_answer_box(img, cm_to_px(1.5 + i*6), cm_to_px(4), str(i*1000))



save_img(img, 'output.png')




<PIL.PngImagePlugin.PngImageFile image mode=1 size=21x21 at 0x19D2C099360>
<PIL.PngImagePlugin.PngImageFile image mode=1 size=21x21 at 0x19D2C09BBE0>
<PIL.PngImagePlugin.PngImageFile image mode=1 size=21x21 at 0x19D2C099BD0>


In [21]:
from PIL import Image, ImageFont, ImageDraw
import cv2
import numpy as np
from pyzbar.pyzbar import decode

def scan_qrcode_from_image_path(pillow_image):
    # Load the image from file using Pillow
    pillow_image = Image.open(image_path)
    

    # Ensure the image is in RGB mode (in case it's RGBA or grayscale)
    if pillow_image.mode != 'RGB':
        pillow_image = pillow_image.convert('RGB')
    
    # Convert Pillow image to OpenCV format (numpy array)
    open_cv_image = np.array(pillow_image)

    # Convert RGB to BGR (OpenCV uses BGR format)
    open_cv_image = open_cv_image[:, :, ::-1]

    # Decode QR codes
    qr_codes = decode(open_cv_image)

    # Create a drawing context
    draw = ImageDraw.Draw(pillow_image)

    qr_code_coords = []

    # Iterate through detected QR codes
    for qr in qr_codes:
        # Get the data from the QR code
        qr_data = qr.data.decode('utf-8')
        # print(f"QR Code Data: {qr_data} {qr.polygon}")
        

        # Get the bounding box (rectangle)
        rect_points = qr.rect  # This is a tuple (x, y, width, height)
        top_left = (rect_points[0], rect_points[1])
        bottom_right = (rect_points[0] + rect_points[2], rect_points[1] + rect_points[3])

        qr_code_coords.append({
            "coords": [top_left, bottom_right],
            "data": qr_data
        })
        
        # Draw the rectangle (color: green, thickness: 3)
        draw.rectangle([top_left, bottom_right], outline="green", width=3)
        
        # Display the decoded QR code data on the image
        font = ImageFont.load_default()
        text_position = (top_left[0], top_left[1] - 10)  # Place text above the rectangle
        draw.text(text_position, qr_data, fill="green", font=font)
        

    return qr_code_coords

def cut_image(cv2_img, top_left, bottom_right):
    return cv2_img[top_left[1]:bottom_right[1] , top_left[0]:bottom_right[0]]

# Specify the path to your image file
image_path = "output.png"
pillow_image = Image.open(image_path)

# Scan QR codes from the image file
data = scan_qrcode_from_image_path(pillow_image)

# def cut_img_from_qr_data(img, data):
    
    
#     for index, qr_code in enumerate(data):
#         data[index]["image"] = 


print(data)



[{'coords': [(1502, 1329), (1619, 1446)], 'data': '2000'}, {'coords': [(83, 1015), (200, 1132)], 'data': '2000'}, {'coords': [(1502, 857), (1619, 974)], 'data': '1000'}, {'coords': [(83, 543), (200, 660)], 'data': '1000'}, {'coords': [(1502, 385), (1619, 502)], 'data': '0'}, {'coords': [(83, 71), (200, 188)], 'data': '0'}]
